<title> Simulation of observations with TD in Argentina</title>

<title> Sim </title>

<center><font size=20> <b>Verification of QUBIC noise on TODs and maps</b></font></center>

I want to verify that the noise is propagated correctly from the input defined in the QUBIC dictionary to the final maps. To do this we follow this approach:

1. Get the detector noise spectral density (NEP) in W/sqrt(Hz) from instrument dictionary
1. Convert the NEP into NET in K x sqrt(s) (see below)
1. Generate a sky map to be observed
1. Make and observation with a single detector with ranndom pointing over a defined area, $A$. The observation will have no photon noise, no atmospheric noise, no 1/f noise. The only source of noise will be the detector noise
1. Calculate the TOD standard deviation, $\sigma_\mathrm{TOD}$ and compare with $\mathrm{NET}/\sqrt{\tau}$ where $\tau$ is the integration time
1. Calculate the theoretical average map noise standard deviation, $\sigma_\mathrm{map}^\mathrm{theo}$ that is calculated as follows:
    1. Measure the pixels covered by the observation in the map, $N_\mathrm{pix}$
    1. Calculate the average integration time per pixel, $\tau _\mathrm{pix} = T_\mathrm{obs} / N_\mathrm{pix}$, where $T_\mathrm{obs}$ is the total observation time
    1. Calculate $\sigma_\mathrm{map}^\mathrm{theo} = \sigma_\mathrm{TOD} / \sqrt{h_\mathrm{pix}}$ where $\sigma_\mathrm{TOD}$ is the NET
1. Calculate observed map and subtract signal map
1. Calculate the map standard deviation $\sigma_\mathrm{map}$
1. Compare $\sigma_\mathrm{map}$ with $\sigma_\mathrm{map}^\mathrm{theo}$ 

# Imports and functions

In [ ]:
from __future__ import division
import qubic
import numpy as np
import pylab as pl
import healpy as h
from astropy.io import fits
from pysimulators import FitsArray
from pysm.nominal import models
import pickle

In [ ]:
def build_sky_map(instrument_dict, \
                  sky_config, \
                  nf_sub, \
                  map_filename):
    '''
    This function builds a sky map given an instrument dictionary and a sky configuration and writes the map in a fits file

    INPUTS
    instrument_dict - STRING - the location of the instrument configuration dictionary
    sky_config      - DICT   - the sky configuration (see https://pysm-public.readthedocs.io/en/latest/models.html for more info)
    nf_sub          - INT    - Number of sub-bands (main band defined in instrument dictionary). Must be geq 2
    map_filename    - STRING - the filename of the output map (without the .fits suffix)

    OUTPUTS
    out_filename    - STRING - the full output filename    
    '''
    
    import os

    from pysm.nominal import models

    import qubic
    from qubic import SpectroImLib as si

    from pysimulators import FitsArray

    d = qubic.qubicdict.qubicDict()
    d.read_from_file(instrument_dict)

#    nf_sub = [1,]#[2, 4, 5, 10, 12, 14, 15, 16, 18, 20, 22, 24]
    dirc = './'

    try:
        os.makedirs(dirc)
    except:
        pass

    out_fname = []
    for nf in nf_sub:
        print(nf)
        d['nf_sub'] = nf
        #sky_config = {'dust': models('d1', d['nside']), 'cmb': models('c1', d['nside'])}
        #sky_config = {'cmb': models('c1', d['nside'])}

        Qubic_sky = si.Qubic_sky(sky_config, d)
        x0 = Qubic_sky.get_simple_sky_map()
        dirc2 = dirc
        try:
            os.makedirs(dirc2)
        except:
            pass
        
        fname = '%s_Nside_%i_Nfreq_%i.fits' % (map_filename, d['nside'], nf)
        FitsArray(x0).save(fname)
        out_fname.append(fname)
    
    return out_fname
    

In [ ]:
def select_detector_subset(instrument, detectors):
    '''
    This function allows one to select a detector subset for the simulation

    INPUTS
    instrument - qubic.instrument.QubicMultibandInstrument - the MultiInstrument object
    detectors  - a list of integers or the string 'All'

    OUTPUTS
    out -qubic.instrument.QubicMultibandInstrument -_The updated MultiInstrument object
    '''
    if detectors == 'All':
        out = instrument
    else:
        for i in range(len(instrument)):
            instrument[i].detector = instrument[i].detector[detectors]
        out = instrument
    return out

In [ ]:
def build_qubic_map(sky_map, tod, cov, inst_dict_fn, detectors):
    '''
    This function builds a map observed by QUBIC given a sky map and instrument dictionary and a detectors lis

    INPUTS
    sky map - np.array(nf, npix, 3) - nf is the number of frequencies, npix is the number of pixels and 3 is for IQU maps
    tod        - NP.ARRAY(nf_rec, len)     - the TOD. nf_rec is the number of reconstructed sub-bands, len is the length of the TOD
    cov        - NP.ARRAY(npix)            - the coverage map
    inst_dicg_fn - STRING - filename of the instrument dictionary
    detectors - LIST - a list of integers or the string 'All'

    OUTPUTS
    maps_recon - NP.ARRAY(nf_rec, npix, 3) - the reconstructed map. nf_rec is the number of reconstracted sub-bands, npix is the number of pixels, 3 is for IQU maps
    '''

 # Define sky configuration
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(inst_dict_fn)
    
    # Pointing strategy
    p = qubic.get_pointing(d)
    print('=== Pointing DONE! ===')

    # Model of the scene at the time of observation
    s = qubic.QubicScene(d)

    # Create MultibandInstrument. Model of the instrument with a focal plane sensitivity to 
    # multiple frequencies
    q = select_detector_subset(qubic.QubicMultibandInstrument(d), detectors)

    # Generate the sub-band limits to be used (later we can see what information we are ignoring with '_')
    _, nus_edge_in, _, _, _, _ = qubic.compute_freq(d['filter_nu'] / 1e9, d['nf_sub'],
                                                    d['filter_relative_bandwidth'])
    # Finally create an acquisition multiband operator which combines all relevant information
    #scene, instrument configuration and strategy pointing. 
    a = qubic.QubicMultibandAcquisition(q, p, s, d, nus_edge_in)    

    #Coverage map
    # Map-making process for a given number of reconstructed subbands
    nf_sub_rec = d['nf_recon'][0]

    print('************* Map-Making on {} sub-map(s) *************'.format(nf_sub_rec))

    ## Build a MultiAcquisition operator for reconstruction (map-making). As it is the inverse
    # process but seen with 'different' instrument (one with nf_rec channels).  
    # Recompute the edges of sub-bands :
    _, nus_edge, _, _, _, _ = qubic.compute_freq(d['filter_nu'] / 1e9, nf_sub_rec, d['filter_relative_bandwidth'])

    # Create a MultiBandAcquisition
    arec = qubic.QubicMultibandAcquisition(q, p, s, d, nus_edge)
        
    # From TOD reconstruct sky maps.
    maps_recon, nit, error = arec.tod2map(tod, d, cov=cov)

    print('The shape of the reconstructed maps is (nf_recon, #pixels, #stokes) :', maps_recon.shape)
    print('{} iterations were needed to get an error of {}'.format(nit, error))    
    
    return maps_recon

In [ ]:
def build_qubic_tod(sky_map, inst_dict_fn, detectors):
    '''
    This function builds a TOD observed by QUBIC given a sky map and instrument dictionary and a detectors list

    INPUTS
    sky map - np.array(nf, npix, 3) - nf is the number of frequencies, npix is the number of pixels and 3 is for IQU maps
    inst_dicg_fn - STRING - filename of the instrument dictionary
    detectors - LIST - a list of integers or the string 'All'

    OUTPUTS
    TOD        - NP.ARRAY(nf_rec, len)     - the TOD. nf_rec is the number of reconstructed sub-bands, len is the length of the TOD
    '''

    # Define sky configuration
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(inst_dict_fn)
    
    # Pointing strategy
    p = qubic.get_pointing(d)
    print('=== Pointing DONE! ===')

    # Model of the scene at the time of observation
    s = qubic.QubicScene(d)

    # Create MultibandInstrument. Model of the instrument with a focal plane sensitivity to 
    # multiple frequencies
    q = select_detector_subset(qubic.QubicMultibandInstrument(d), detectors)

    # Generate the sub-band limits to be used (later we can see what information we are ignoring with '_')
    _, nus_edge_in, _, _, _, _ = qubic.compute_freq(d['filter_nu'] / 1e9, d['nf_sub'],
                                                    d['filter_relative_bandwidth'])
    # Finally create an acquisition multiband operator which combines all relevant information
    #scene, instrument configuration and strategy pointing. 
    a = qubic.QubicMultibandAcquisition(q, p, s, d, nus_edge_in)    

    #TOD
    # MultiFreq TOD making
    #(i.e. TOD for several subbands to reproduce a wideband observation)
    TOD, maps_convolved_useless = a.get_observation(sky_map, noiseless=d['noiseless'])#, convolution = True)
    del(maps_convolved_useless)
    
    return TOD

In [ ]:
def build_qubic_cov(sky_map, inst_dict_fn, detectors):
    '''
    This function builds a coverage map observed by QUBIC given a sky map and instrument dictionary and a detectors list

    INPUTS
    sky map - np.array(nf, npix, 3) - nf is the number of frequencies, npix is the number of pixels and 3 is for IQU maps
    inst_dict_fn - STRING - filename of the instrument dictionary
    detectors - LIST - a list of integers or the string 'All'

    OUTPUTS
    cov        - NP.ARRAY(npix)            - the coverage map
    '''

 # Define sky configuration
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(inst_dict_fn)
    
    # Pointing strategy
    p = qubic.get_pointing(d)
    print('=== Pointing DONE! ===')

    # Model of the scene at the time of observation
    s = qubic.QubicScene(d)

    # Create MultibandInstrument. Model of the instrument with a focal plane sensitivity to 
    # multiple frequencies
    q = select_detector_subset(qubic.QubicMultibandInstrument(d), detectors)

    # Generate the sub-band limits to be used (later we can see what information we are ignoring with '_')
    _, nus_edge_in, _, _, _, _ = qubic.compute_freq(d['filter_nu'] / 1e9, d['nf_sub'],
                                                    d['filter_relative_bandwidth'])
    # Finally create an acquisition multiband operator which combines all relevant information
    #scene, instrument configuration and strategy pointing. 
    a = qubic.QubicMultibandAcquisition(q, p, s, d, nus_edge_in)    

    #Coverage map
    # Map-making process for a given number of reconstructed subbands
    nf_sub_rec = d['nf_recon'][0]

    print('************* Map-Making on {} sub-map(s) *************'.format(nf_sub_rec))

    ## Build a MultiAcquisition operator for reconstruction (map-making). As it is the inverse
    # process but seen with 'different' instrument (one with nf_rec channels).  
    # Recompute the edges of sub-bands :
    _, nus_edge, _, _, _, _ = qubic.compute_freq(d['filter_nu'] / 1e9, nf_sub_rec, d['filter_relative_bandwidth'])

    # Create a MultiBandAcquisition
    arec = qubic.QubicMultibandAcquisition(q, p, s, d, nus_edge)
    
    # Get coverage map
    # i.e. how many times were seen each pixel of the sky (not trivial because of the synthetic beam)
    cov = arec.get_coverage()
        
    return cov

In [ ]:
def nep2net(nep, freq, bandwidth, temp):
    h = 6.62607004e-34
    k = 1.38064852e-23
    x = h*freq/k/temp
    fact = 1/(2*k*bandwidth)*(np.exp(x)-1)**2/(x**2*np.exp(x))
    net = nep * fact
    return net
    

# Get detector noise from dictionary and convert to K*sqrt(s)

In [ ]:
# Define folders
root_folder = '/Users/hamilton/Qubic/ScienceCaseTD/DebugCodeDaniele'
data_folder = root_folder+'/data_files/'
maps_folder = root_folder+'/maps/'
#inst_dict_fn = '%sTD_spectroimaging.dict' % (data_folder)
inst_dict_fn = 'TD_spectroimaging.dict'

# Read dictionary
d = qubic.qubicdict.qubicDict()
d.read_from_file(inst_dict_fn)

# Read noise
nep = d['detector_nep']
print('Detector NEP = %1.2e W/sqrt(Hz)' % nep)

In [ ]:
# Convert to NET
nu0 = d['filter_nu']
bw = nu0 * d['filter_relative_bandwidth']
temp = d['temperature']
net = nep2net(nep, nu0, bw, temp)
tau = d['period']
print('Center frequency = %3.0f GHz' % (nu0 / 1e9))
print('Bandwidth = %3.0f GHz' % (bw / 1e9))
print('Integration time = %1.3f s' % tau)
print('Detector NET = %1.2e K x sqrt(s)' % net)

# Generate sky map to be observed

<p style="font-size:20px"> 
Here I generate the input sky map for the observation. I include CMB, synchrotron and dust with the symplest model in PySm (s1, d1, see details in <a href="https://pysm-public.readthedocs.io/en/latest/models.html">https://pysm-public.readthedocs.io/en/latest/models.html</a>.
</p>

In [ ]:
# Define sky configuration
sky_config = {'synchrotron': models('s1', d['nside']),'dust': models('d1', d['nside']), 'cmb': models('c1', d['nside'])}

# Define number of sub-bands
nf_sub = [2,] #[2, 4, 5, 10, 12, 14, 15, 16, 18, 20, 22, 24]

# Define output map filename
out_map = '%ssyn_dust_cmb_map.fits' % (maps_folder)

# Define center freq, band extremes and sub_bands
nu_min      = nu0 - bw / 2.
nu_max      = nu0 + bw / 2.
delta       = bw / nf_sub[0]
freq_array  = np.array([nu_min + (j-1/2.) * delta for j in np.arange(1,nf_sub[0]+1)])

In [ ]:
outmap = build_sky_map(inst_dict_fn, sky_config, nf_sub, out_map)

In [ ]:
mp = FitsArray(outmap[0])

<p style="font-size:20px"> 
Display the generated maps</a>.
</p>

In [ ]:
fig = pl.figure(figsize = (26,26))
fig.suptitle('Input sky map', fontsize = 'xx-large')
ind = 0
indices = np.arange(len(freq_array))
for fr, index in zip(freq_array, indices):
    ind = ind + 1
    ax = fig.add_subplot(nf_sub[0], 3,ind)
    h.visufunc.mollview(mp[index, :, 0], hold = True, title = '%3.1f GHz I' % (fr/1.e9))

    ind = ind + 1
#    print(fr/1e9, ind)
    ax = fig.add_subplot(nf_sub[0], 3,ind)
    h.visufunc.mollview(mp[index, :, 1], hold = True, title = '%3.1f GHz Q' % (fr/1.e9))

    ind = ind + 1
#    print(fr/1e9, ind)
    ax = fig.add_subplot(nf_sub[0], 3,ind)
    h.visufunc.mollview(mp[index, :, 2], hold = True, title = '%3.1f GHz U' % (fr/1.e9))
pl.show()
#pl.savefig(maps_folder + 'syn_dust_cmb_map.pdf',bbox_inches = 'tight')
pl.close('all')

# Make observation and check noise on TOD

Now I build a TOD with 240000 pointings sampled at 1 Hz with and integration time of 1 second. I consider only one detector

In [ ]:
y2sec = 24*365*3600.
eff_duration = d['npointings'] * d['period']/y2sec
detector = [0]
print(eff_duration)

In [ ]:
tod = build_qubic_tod(mp, inst_dict_fn, detector)

In [ ]:
pl.plot(tod[0,:])

Now I calculate the standard deviation of the TOD, $\sigma$. Then I should get $\sigma = \mathrm{NEP}/\sqrt{2}/\sqrt{\tau}$

In [ ]:
sigma = np.std(tod[0,:])
print('sigma measured = %1.3e W, sigma theo = %1.3e W' %(sigma, nep/np.sqrt(2)/np.sqrt(tau)))

# Check noise on the map

## Define coverage

In [ ]:
# Build coverage map and select pixels with good coverage
threshold = 0.1
cov = build_qubic_cov(mp, inst_dict_fn, [0])
okpix = cov[0,:] > (threshold * np.max(cov[0,:]))

In [ ]:
h.mollview(cov[0,:])

## Build observed map

In [ ]:
# Build map
qubic_mp = build_qubic_map(mp, tod, cov, inst_dict_fn, [0])

In [ ]:
# Set the unvovered pixels to UNSEEN and select the intensity map
for i in np.arange(3):
    qubic_mp[~okpix,i] = h.UNSEEN
qubic_I = qubic_mp[:,0]

In [ ]:
h.mollview(qubic_I)

## Calculate theoretical rms on map

Now we calculate the theoretical (average) rms on the map. This is equal to:
$\sigma_\mathrm{map} = \sigma_\mathrm{tod}/\sqrt{\tau_\mathrm{m}}$, where $\sigma_\mathrm{tod}$ is the TOD rms in $\mathrm{K}\times\sqrt{s}$ and $\tau_\mathrm{m}$ is the average integration time per pixel. 

We can estimate $\tau_\mathrm{m}$ as $T_\mathrm{obs}/N_\mathrm{pix}$, where $T_\mathrm{obs}$ is the total observation time and $N_\mathrm{pix}$ is the number of observed pixels. 

In [ ]:
Tobs = sum(cov[0,okpix]) * d['period']
Npix = len(qubic_I[okpix])
tau_m = Tobs/Npix
sigma_map = net / np.sqrt(tau_m)

In [ ]:
print('Total observation time: %6.2f seconds' % Tobs)
print('Npix: %i' % Npix)
print('Average tau: %3.2f seconds' % tau_m)
print('Theoretical sigma_map: %2.3f microK' % (sigma_map * 1e6))

## Measure average rms on map

In [ ]:
# Now I produce noise-only maps by subtracting the signal maps and define the same mask as the QUBIC one
sky_I =0.5*(mp[0,:,0]+mp[1,:,0])
sky_I[~okpix] = h.UNSEEN

In [ ]:
h.mollview(sky_I)

In [ ]:
diffmap = qubic_I - sky_I
diffmap[~okpix] = h.UNSEEN

In [ ]:
h.mollview(diffmap)

In [ ]:
print("Measured sigma on map: %5.3f microK" % np.std(diffmap[okpix]))

<font size = 16>It is about 3 orders of magnitudes larger than the estimate</font>